### Data prep
1. Prepare the data in date index and one column with all categories and a column with costs
2. Prepare the data in date index and one column for each category with costs

### Plotting
1. Line plot with datetime on x-axis
2. stacked bar plot with datetime on x-axis

### Widgets
1. Set up widgets for catogories and time
2. Set up a select widget and present in a table

### Good stuff
- plot.xaxis.major_label_orientation = -pi/4
- x_axis_type='datetime'
- [str(x) for x in sorted(set(df['yearMth']))]


### Status
1. sel1_modify2.py scatter plot of costs vs date. Select points which are shown in a DataTable Working
2. test2.py stacked bar chart on yyyyMM with stacked values of categories, callback almost working
   - adjust bars after slider - almost OK
   - update yyyyMM in x-range - almost OK
   - when sliding from left -> right it looks good, but reversing make it show wrong bars

2. Stacked bar chart seems hard to make, fix with a function (?) 
   - I need both to have both on_change method (possible only with vbar) and grouped bar on categorical x-axis (possible only with Bar) 
   - Modify ## X ##
3. Try creating a line plot split by category and a total


In [1]:
import pandas as pd

from bokeh.io import curdoc
from bokeh.layouts import row, column, layout
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import PreText, Select, Slider
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.models.callbacks import CustomJS
from bokeh.charts import Area

# import data
fname = 'data/Export.csv'

#df = pd.read_csv(fname, header=1, parse_dates=['Bokforingsdatum'])
#df = data.set_index('Bokforingsdatum')
df = pd.read_csv(fname, sep=";")
#df = dfa.set_index('Bokforingsdatum')

df['dt'] = pd.to_datetime(df['Bokforingsdatum'])
# df["dt"].dt.month

df['Belopp'] = df['Belopp'].str.replace(',','.')
df['Belopp'] = pd.to_numeric(df['Belopp'])
df['Cost'] = -1*df['Belopp']
df['yearMth'] = df['dt'].map(lambda x: 100*x.year + x.month)

df = df.sort_values(by=["yearMth","HASH"])
#df = df[df['Belopp']>0]


# define source data
source = ColumnDataSource(df)

/Users/tomas/miniconda2/envs/py27_nb/lib/python2.7/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
The bokeh.charts API has moved to a separate 'bkcharts' package.

This compatibility shim will remain until Bokeh 1.0 is released.
After that, if you want to use this API you will have to install
the bkcharts package explicitly.

  warn(message)


In [7]:
from bokeh.plotting import figure, show
#_df = df[df.HASH == 'TOMAS']

_df = df
output_notebook()
_df['idx'] = range(len(_df))
p = figure(title="line", plot_width=300, plot_height=300)
p.line(x=_df.idx, y=_df.Cost)
show(p)
#df = df.set_index('yearMth')



Loading BokehJS ...

## Plotting
I need both to have both on_change method (possible only with vbar) and grouped bar on categorical x-axis (possible only with Bar) 

In [53]:
# OK vbar_stack
# create a dict with 
# category : [value for each time event]
# time : [each timestamp]
# p.vbar_stack(category, x='time', width=0.9, color=colors, source=source,
#             legend=[x for x in category])
#
#  TODO: check if pivot must be aggregated on mth year level, maybe dt is too granualar. Plot might look ok
#---
from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from math import pi
import numpy as np

# pivot data
df_pivot2 = df.pivot_table(values='Cost', index=['HASH'],aggfunc=np.sum, columns="yearMth").fillna(0)
df_pivot2 = df_pivot2.reset_index(drop=False)

category = [c for c in sorted(set(df['HASH']))]
#category = ['CAR','FOOD']
colors = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78','#2ca02c','#98df8a','#d62728','#ff9896','#9467bd','#c5b0d5','#8c564b']
# since yearMth is index in the pivot, I'm taking the values from the original df <redo>
dates = [str(a) for a in sorted(set(df.yearMth))]

wtf = {'time' : dates}

for i in range(len(category)):
    name = df_pivot2.iloc[i:i+1,:1].get_value(i,'HASH')
    values = df_pivot2.iloc[i+0:i+1,1:].get_values()
    val = values[0,:].tolist()
    wtf.update({name : val})
    
sourceW = ColumnDataSource(data=wtf)

p = figure(x_range=dates, plot_height=400, plot_width=800, title="by Year",
           toolbar_location=None, tools="")

p.vbar_stack(category, x='time', width=0.4, legend=[value(x) for x in category], color = colors, source=sourceW)


# format plot
p.xaxis.major_label_orientation = -pi/4
p.y_range.start = -1000
p.x_range.range_padding = 0.5
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "vertical"
p.legend.label_width = 5
p.legend.label_text_font_size = '6pt'
p.background_fill_color = "white"
p.background_fill_alpha = 0.25

p.border_fill_color = "whitesmoke"
p.min_border_left = 80

output_notebook()

show(p)

Loading BokehJS ...

In [7]:
from bokeh.charts import Bar
from bokeh.models.widgets import CheckboxGroup, CheckboxButtonGroup
import numpy as np
from bokeh.layouts import row, widgetbox

output_notebook()
#output_file('dash.html')


selection = ['TOMAS', 'CAR']
#bar = Bar(df[(df['HASH'].isin(selection))], label='yearMth', values='Belopp', agg='sum', stack='HASH', title="Ex", tools = 'hover')

bar = Bar(source.data, label='yearMth', values='Belopp', agg='sum', stack='HASH', title="Expec")

show(bar)



Loading BokehJS ...

TypeError: 'NoneType' object is not iterable

In [27]:
from bokeh.models.widgets import CheckboxGroup, CheckboxButtonGroup
import numpy as np
from bokeh.layouts import row, widgetbox
from bokeh.models.glyphs import VBar
from bokeh.models import ColumnDataSource, DataRange1d, Plot, LinearAxis, Grid
from bokeh.io import curdoc, show
from bokeh.models.ranges import FactorRange
from math import pi

output_file('vbar.html')
x_label = "Category"
y_label = "Amount"
title = "Test bar plot"

plot = figure(plot_width=900, plot_height=300,
              x_axis_label=x_label,
              y_axis_label=y_label,
#              x_axis_type='datetime',
              title=title,
#              x_range=FactorRange(factors=list(set(df['HASH'])))
#               x_range=FactorRange(factors=[str(x) for x in list(set((df['yearMth'])))])
                x_range=FactorRange(factors=[str(x) for x in sorted(set(df['yearMth']))])
              
              )
plot.xaxis.major_label_orientation = -pi/4


#plot.vbar(source=source,x="HASH", top="Belopp",bottom=0,width=0.3)
plot.vbar(source=source,x="yearMth", top="Belopp",bottom=0,width=0.3)
#plot.vbar_stack('HASH', x='yearMth', width=0.9, source=source)


show(plot)

In [9]:
[str(x) for x in sorted(set(df['yearMth']))]

['201508',
 '201509',
 '201510',
 '201511',
 '201512',
 '201601',
 '201602',
 '201603',
 '201604',
 '201605',
 '201606',
 '201607',
 '201608',
 '201609',
 '201610',
 '201611',
 '201612',
 '201701',
 '201702',
 '201703',
 '201704',
 '201705',
 '201706',
 '201707',
 '201708',
 '201709']

In [56]:
from bokeh.layouts import column
from bokeh.plotting import figure, show ,output_notebook
from bokeh.models import ColumnDataSource, Select
from bokeh.models.callbacks import CustomJS
from bokeh.models import PrintfTickFormatter

output_notebook()    

source=ColumnDataSource(data=df) 

p1 = figure(plot_width=900, plot_height=300, logo=None, toolbar_location='right',x_axis_type='datetime', tools='hover') 
p1.circle(source=source, x='dt', y='Belopp', size=10, alpha=0.5)  


p1.background_fill_color = "white"
p1.background_fill_alpha = 0.25

p1.border_fill_color = "whitesmoke"
p1.min_border_left = 80
p1.xaxis.axis_label = "Date"
p1.yaxis.axis_label = "Cost"
#p1.xaxis[0].formatter = PrintfTickFormatter(format="%4.1e")
p1.yaxis[0].formatter = PrintfTickFormatter(format="%5.0f kr")
# change just some things about the y-grid
#p1.ygrid.band_fill_alpha = 0.1
#p1.ygrid.band_fill_color = "olive"

show(p1)


Loading BokehJS ...

## Callbacks

In [2]:
# select all rows selected in HASH column
def update_plot(attrname, old, new):
    data = source.data
    o_data = source.data
    switch = cb_group.active
    items = [SEL[i] for i in active]
    for i in range(data['HASH'].length):
        if (data['HASH'][i] in str(items)):
            data[key].push(o_data[key][i])
    update_graph(data)
                
def update_graph(data):
    bar = Bar(data, label='yearMth', values='Cost', agg='sum', stack='HASH', title="Expected Sales by year", tools = 'hover')


## Widgets

In [3]:
#cb_group = CheckboxButtonGroup(labels=['Close', 'Adj Close'],active=[0,1])

# define widgets
SEL = [x for x in sorted(set(df['HASH']))]
#cb_group = CheckboxGroup(labels = SEL, active = np.ones(len(SEL),dtype=np.int).tolist())

cb_group = CheckboxGroup(labels = SEL, active = [x for x in range(len(SEL))])

cb_group.on_change(update_plot)


inputs=row(cb_group,bar)

show(inputs)



NameError: name 'CheckboxGroup' is not defined

## Testing

In [4]:
df.columns

Index([u'Bokforingsdatum', u'Valutadatum', u'Verifikationsnummer',
       u'Text / mottagare', u'Belopp', u'Saldo', u'RND', u'LEFT(D6;4)',
       u'HASH', u'dt', u'Cost', u'yearMth'],
      dtype='object')

In [28]:
import numpy as np

from bokeh.models import ColumnDataSource, DataRange1d, Plot, LinearAxis, Grid
from bokeh.models.glyphs import VBar
from bokeh.io import curdoc, show

N = 9
x = np.linspace(-2, 2, N)
y = x**2

source = ColumnDataSource(dict(x=x,top=y,))

xdr = DataRange1d()
ydr = DataRange1d()

plot = Plot(
    title=None, x_range=xdr, y_range=ydr, plot_width=300, plot_height=300,
    h_symmetry=False, v_symmetry=False, min_border=0, toolbar_location=None)

glyph = VBar(x="x", top="top", bottom=0, width=0.5, fill_color="#b3de69")
plot.add_glyph(source, glyph)

xaxis = LinearAxis()
plot.add_layout(xaxis, 'below')

yaxis = LinearAxis()
plot.add_layout(yaxis, 'left')

plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))

curdoc().add_root(plot)

show(plot)

In [ ]:
a = df.iloc[[0]]
a
b = [a, df.iloc[[1]]]
res2 = pd.concat([res2, df.iloc[[1]]])
res2
a

In [ ]:
combined_callback_code = """
var data = source.get('data');
var original_data = original_source.get('data');
var grp = grp_select_obj.get('value');
console.log("grp: " + grp);
for (var key in original_data) {
    data[key] = [];
    for (var i = 0; i < original_data['grp'].length; ++i) {
        if (( grp === 'ALL' || original_data['grp'][i] === grp)) {
            data[key].push(original_data[key][i]);
        }
    }
}
target_obj.trigger('change');
source.trigger('change');

"""

# now define the callback objects now that the filter widgets exist
generic_callback = bokeh.models.CustomJS(
    args=dict(source=source, 
              original_source=original_source, 
              grp_select_obj= [SEL[i] for i in switch], 
              target_obj=data_table),
    code=combined_callback_code
)



In [ ]:
active=[0,1,2]
SEL = [x for x in sorted(set(df['HASH']))]
item = [SEL[i] for i in active]
df['HASH'][1] in str(item)
item

In [ ]:
df_pivot = df.pivot_table(values='Belopp', index=['dt'], columns="HASH").fillna(0)
df_pivot = df_pivot.reset_index(drop=False)

print df_pivot.shape
df_pivot.head(5)


In [6]:
from bokeh.layouts import layout
from bokeh.plotting import output_file


output_file('dashboard.html')

key = 'HOUSE'
foo = df_pivot[df_pivot[key]<0]
plot = figure( tools='', toolbar_location=None,title="Costs",x_axis_type="datetime")

plot.line(foo['dt'], -1*foo[key],  line_width=3, line_alpha=0.6, color=color, legend = key)

fruits = [c for c in foo.columns if c not in ['HASH','dt']]
years = [str(y) for y in sorted(set(foo['dt'].dt.year)) ]

#colors = ["#c9d9d3", "#718dbf", "#e84d60"]

foo['year'] = foo['dt'].dt.year
foo = foo.drop('dt', 1)

source = ColumnDataSource(data=foo)


p = figure(x_range=fruits, plot_height=350, title="Fruit Counts by Year",
           toolbar_location=None, tools="")

p.vbar_stack(years, x='fruits', width=0.9, color='r', source=source,
             legend=[x for x in years])

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

l = layout(p,plot)
show(l)

#show(plot)


NameError: name 'df_pivot' is not defined

In [ ]:
data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 4, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}
source = ColumnDataSource(data=foo[fruits])


type(source)
source.data
foo['year'] = foo['dt'].dt.year
foo = foo.drop('dt', 1)


In [ ]:
df_pivot2 = df.pivot_table(values='Belopp', index=['HASH'],aggfunc=np.sum, columns="yearMth").fillna(0)
df_pivot2 = df_pivot2.reset_index(drop=False)
print df_pivot2.shape
df_pivot2.head(50)

In [ ]:
df_test = df[df['Belopp']<0].groupby([df['yearMth'],'HASH'])['Belopp'].sum()
df_test.columns

In [ ]:
plot = figure( tools='', toolbar_location=None,title="Costs",x_axis_type="datetime")

plot.line(df_test['HASH'], -1*df_plot['Belopp'],  line_width=3, line_alpha=0.6, color=color, legend = key)



In [ ]:
# define categories based on content in the data
DEFAULT_TICKERS = [x for x in sorted(set(df['HASH']))]

In [ ]:
# define widgets
_select = Select(value = 'ALL', options = DEFAULT_TICKERS)
_slideM = Slider(start=1, end=12, value=1, step=1, title="Month", callback=callback)
_slideY = Slider(start=2015, end=2017, value=2015, step=1, title="Year", callback=callback)

callback.args["mth"] = _slideM
callback.args["yr"] = _slideY

In [29]:
# define plots

# select a palette
from bokeh.palettes import Dark2_5 as palette

# itertools handles the cycling
import itertools  

# create a color iterator
colors = itertools.cycle(palette) 

plot = figure( tools='', toolbar_location=None,title="Costs",x_axis_type="datetime")
yr = 2017
mth
for key, color in itertools.izip(DEFAULT_TICKERS, colors):
    df_plot = df[(df['Belopp'] < 0) & (df['HASH'] == key) & (df['dt'].dt.year == yr) & (df['dt'].dt.month > mth)]
    plot.line(df_plot['dt'], -1*df_plot['Belopp'],  line_width=3, line_alpha=0.6, color=color, legend = key)



show(plot)

NameError: name 'mth' is not defined

In [ ]:
dates = [str(x) for x in sorted(set(df['dt'].dt.year))]
dates

In [ ]:
source.data

In [ ]:
#OK#
output_notebook()


fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ["2015", "2016", "2017"]
colors = ["#c9d9d3", "#718dbf", "#e84d60"]

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 4, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

source = ColumnDataSource(data=data)

p = figure(x_range=fruits, plot_height=350, title="Fruit Counts by Year",
           toolbar_location=None, tools="")
from bokeh.core.properties import value


p.vbar_stack(years, x='fruits', width=0.9, color=colors, source=source,legend=[value(l) for l in years])

show(p)

In [ ]:
# OK# 
# Try and modify this
## X ##

from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

output_file("stacked.html")

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ["2015", "2016", "2017"]
colors = ["#c9d9d3", "#718dbf", "#e84d60"]

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 4, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

source = ColumnDataSource(data=data)

p = figure(x_range=fruits, plot_height=250, title="Fruit Counts by Year",
           toolbar_location=None, tools="")

p.vbar_stack(years, x='fruits', width=0.9, color=colors, source=source,
             legend=[value(x) for x in years])

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)

In [ ]:
category = [c for c in sorted(set(df['HASH']))]
#category = ['CAR','FOOD']
colors = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78','#2ca02c','#98df8a','#d62728','#ff9896','#9467bd','#c5b0d5','#8c564b','#c49c94','#e377c2','#f7b6d2','#7f7f7f','#c7c7c7','#bcbd22','#dbdb8d','#17becf','#9edae5']
# since yearMth is index in the pivot, I'm taking the values from the original df <redo>
dates = [str(a) for a in sorted(set(df.yearMth))]

print len(category), len(colors), len(dates)

In [ ]:
i=1

name = df_pivot2.iloc[i:i+1,:1].get_value(i,'HASH')
values = df_pivot2.iloc[i+0:i+1,1:].get_values()
val = values[0,:].tolist()
   
name

[str(a) for a in sorted(set(df.yearMth))]

In [ ]:
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show, output_notebook
from bokeh.palettes import brewer
output_notebook()

N = 20
categories = ['y' + str(x) for x in range(10)]
data = {}
data['x'] = np.arange(N)
for cat in categories:
    data[cat] = np.random.randint(10, 100, size=N)

df = pd.DataFrame(data)
df = df.set_index(['x'])

def stacked(df, categories):
    areas = dict()
    last = np.zeros(len(df))
    for cat in categories:
        next = last + df[cat]
        areas[cat] = np.hstack((last[::-1], next))
        last = next
    return areas

_cats = ['y0', 'y1', 'y2']

areas = stacked(df, _cats)

colors = brewer["Spectral"][len(areas)]

x2 = np.hstack((data['x'][::-1], data['x']))

#p = figure(x_range=(0, 19), y_range=(0, 800))
p = figure()
p.grid.minor_grid_line_color = '#eeeeee'

p.patches([x2] * len(areas), [areas[cat] for cat in _cats],
          color=colors, alpha=0.8, line_color=None, legend = _cats[0])


show(p)


In [ ]:
m = 2
key = 'HOUSE'
df_plot = df[(df['Belopp'] < 0) & (df['HASH'] == 'HOUSE') & (df['dt'].dt.month < m)]
plot.line(df_plot['dt'], -1*df_plot['Belopp'],  line_width=3, line_alpha=0.6, color=color, legend = 'key')

show(plot)

In [ ]:
callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var m = mth.value;
    var y = yr.value;
    x = data['x']
    y = data['y']
    for (var key in data) {
        data[key] = [];
        for (var i = 0; i < data['dt'].length; ++i) {
            if (( data['dt'][i] < m)) {
                data[key].push(original_data[key][i]);
            }
        }
    }
source.change.emit();
""")

In [ ]:
from bokeh.plotting import curdoc, output_file
from bokeh.models.widgets import Button, CheckboxGroup
from bokeh.layouts import widgetbox, row
from bokeh.models import ColumnDataSource, Callback

output_notebook()
states_list = ["Alabama", "Alaska ", "Arizona", "Arkansas"]

states = CheckboxGroup(
        labels = states_list,
        active=[0,1,2,3])

select_all = Button(label="select all")

def update():
    states.active = list(range(len(states_list)))
select_all.on_click(update)

group = widgetbox(select_all, states)

layout = row(group)

show(layout)

In [ ]:
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ["2015", "2016", "2017"]
colors = ["#c9d9d3", "#718dbf", "#e84d60"]

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 4, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

data2 = pd.DataFrame(data)
data2

In [ ]:
help(plot.line)

In [ ]:
sX = ColumnDataSource(df[(df['HASH'] == 'HOUSE') & (df['Belopp']>0)])

sY = ColumnDataSource(df[(df['HASH'] == 'CAR') & (df['Belopp']>0)])

plot = figure( tools='', toolbar_location=None,title="Costs",x_axis_type="datetime")
plot.line(source = sX, x = 'dt', y='Cost')
plot.line(source = sY, x = 'dt', y='Belopp')

output_notebook()

show(plot)

In [89]:
# OK vbar_stack
# create a dict with
# category : [value for each time event]
# time : [each timestamp]
# p.vbar_stack(category, x='time', width=0.9, color=colors, source=source,
#             legend=[x for x in category])
#
#---
import pandas as pd

from bokeh.io import curdoc
from bokeh.layouts import row, column, layout
from bokeh.models import ColumnDataSource, TableColumn, DataTable, CustomJS
from bokeh.models.widgets import PreText, Select, Slider
from bokeh.plotting import figure, show, curdoc
from bokeh.models.callbacks import CustomJS
from bokeh.charts import Area
from bokeh.client import push_session
from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from math import pi
import numpy as np

fname = '/Users/tomas/Documents/Notebook/Dashboards/Bokeh/Server Solution/yearly costs/data/Export.csv'

#df = pd.read_csv(fname, header=1, parse_dates=['Bokforingsdatum'])
#df = data.set_index('Bokforingsdatum')
df = pd.read_csv(fname, sep=";")
#df = dfa.set_index('Bokforingsdatum')

df['dt'] = pd.to_datetime(df['Bokforingsdatum'])
# df["dt"].dt.month

df['Belopp'] = df['Belopp'].str.replace(',','.')
df['Belopp'] = pd.to_numeric(df['Belopp'])
df['Cost'] = -1*df['Belopp']
df['yearMth'] = df['dt'].map(lambda x: 100*x.year + x.month)

df = df.sort_values(by=["yearMth","HASH"])
#df = df[df['Belopp']>0]


# define source data
source = ColumnDataSource(df)

# plot all data #
#---------
# pivot data
df_pivot2 = df.pivot_table(values='Cost', index=['HASH'],aggfunc=np.sum, columns="yearMth").fillna(0)
df_pivot2 = df_pivot2.reset_index(drop=False)

#---------
category = [c for c in sorted(set(df['HASH']))]
colors = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78','#2ca02c','#98df8a','#d62728','#ff9896','#9467bd','#c5b0d5','#8c564b']
# since yearMth is index in the pivot, I'm taking the values from the original df <redo>
dates = [str(a) for a in sorted(set(df.yearMth))]

wtf = {'time' : dates}

for i in range(len(category)):
    name = df_pivot2.iloc[i:i+1,:1].get_value(i,'HASH')
    values = df_pivot2.iloc[i+0:i+1,1:].get_values()
    val = values[0,:].tolist()
    wtf.update({name : val})

#---------
sourceW = ColumnDataSource(data=wtf)
original_source = ColumnDataSource(df)

p = figure(x_range=dates, plot_height=500, title="by Year", toolbar_location=None, tools="")
p.vbar_stack(category, x='time', width=0.6, legend=[value(x) for x in category], color = colors, source=sourceW)
# format plot
p.xaxis.major_label_orientation = -pi/4
p.y_range.start = -1000
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "vertical"

# widget
#offset = Slider(title="offset", value=201508, start=201508, end=201712, step=1)
offsetY = Slider(title="offset", value=2015, start=2015, end=2017, step=1)
offsetM = Slider(title="offset", value=01, start=01, end=12, step=1)

# plot subset data #
def update_data(attrname, old, new):
    # Get the current slider values
#    b = offset.value
    bY = offsetY.value
    bM = offsetM.value
    df_tmp = df[(df['yearMth']>=bY) & (df['yearMth']>=bM)]
    print(df_tmp.shape)
    df_pivot3 = df_tmp.pivot_table(values='Cost', index=['HASH'],aggfunc=np.sum, columns="yearMth").fillna(0)
    df_pivot3 = df_pivot3.reset_index(drop=False)
#    print(df_pivot3.shape)
#    category = [c for c in sorted(set(df_tmp['HASH']))]
#    colors = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78','#2ca02c','#98df8a','#d62728','#ff9896','#9467bd','#c5b0d5','#8c564b']
 #   dates = [str(a) for a in sorted(set(df_tmp.yearMth)) if (a >= b)]
#    print (len(dates))
    wtf = {'time' : dates}

    for i in range(len(category)):
        name = df_pivot3.iloc[i:i+1,:1].get_value(i,'HASH')
        values = df_pivot3.iloc[i+0:i+1,1:].get_values()
        val = values[0,:].tolist()
        wtf.update({name : val})
#    print([len(v) for v in wtf.values()])
#   Update the x-range
    p.x_range.factors = dates
# Update data source
#    sourceW.data=wtf
    sourceW.update(data=wtf)
#    sourceW = ColumnDataSource(data=wtf)


offsetY.on_change('value', update_data)



output_notebook()
lay = layout(offsetY,offsetM, p)
show(lay)

Loading BokehJS ...

In [39]:
wdf = pd.DataFrame()
wdf['Date'] = wtf.keys()
wdf['DateValue'] = wtf.values()
wdf

,Date,DateValue
0,TOMAS,"[-15000.0, 0.0]"
1,LANDET,"[0.0, 0.0, 1127.0, 799.0, 0.0, 0.0, 1200.0, 0...."
2,KIDS,"[1458.0, 1458.0, 3038.0, 1914.0, 1458.0, 1488...."
3,SHOPPING,"[0.0, 2462.0, 844.0, 4492.0, 2661.5, 4255.75, ..."
4,FOOD,"[4950.38, 5407.67, 21448.15, 7490.59, 4004.74,..."
5,HOUSE,"[324.0, 3810.85, 9000.44, 2252.39, 12661.54, 4..."
6,LOAN,"[-4515.0, -4515.0, -4015.0, -4285.0, -4285.0, ..."
7,time,"[201610, 201611, 201612, 201701, 201702, 20170..."
8,OTHER,"[0.0, 3000.0, 35792.0, 0.0, 0.0, 792.0, 0.0, 2..."
9,CAR,"[2157.79, 795.18, 693.57, 2619.17, 1342.41, 19..."


In [45]:
#dict((k, v) for k, v in wtf.iteritems() if v)
slider = 201704
for k, v in wtf2.iteritems():
    if (k == 'time'):
        print k,v
        ++i
        print i
    
#wtf2 = wtf
del wtf2['TOMAS'][0:3]
wtf2

time ['201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709']
10


{'CAR': [2157.79,
  795.18,
  693.57,
  2619.17,
  1342.41,
  1909.95,
  1835.59,
  3359.1400000000003,
  721.44,
  1841.66,
  1628.0,
  801.67,
  4351.99,
  0.0,
  69.0,
  3728.46,
  1556.85,
  2093.3900000000003,
  776.98,
  3352.19,
  1783.21,
  1071.37,
  4214.26,
  3139.02,
  4015.69,
  0.0],
 'FOOD': [4950.38,
  5407.67,
  21448.15,
  7490.59,
  4004.74,
  6209.41,
  6237.85,
  4587.08,
  2241.75,
  4239.58,
  -2970.12,
  9961.52,
  5757.7,
  12277.52,
  3202.73,
  12618.56,
  6299.29,
  8791.08,
  6495.59,
  5248.08,
  7271.48,
  12092.83,
  5790.67,
  5175.06,
  8087.47,
  0.0],
 'HOUSE': [324.0,
  3810.85,
  9000.44,
  2252.3900000000003,
  12661.54,
  4756.66,
  3705.48,
  8489.55,
  12124.91,
  8696.42,
  5956.49,
  5099.6,
  3771.67,
  9279.51,
  -35262.27,
  3712.31,
  13110.2,
  5577.63,
  6504.42,
  8938.52,
  17192.54,
  3277.19,
  6415.4,
  45612.6,
  -527.6799999999998,
  9833.8],
 'INSURANCE': [330.0,
  330.0,
  330.0,
  330.0,
  330.0,
  340.0,
  340.0,
  340.0,
  3

In [46]:
print df.shape
print slider
print df[df['yearMth']>slider].shape

(703, 12)
201704
(128, 12)


In [74]:
def update_data(b):

    # Get the current slider values
   # b = 201606
    df_tmp = df[df['yearMth']>b]
    df_pivot2 = df_tmp.pivot_table(values='Cost', index=['HASH'],aggfunc=np.sum, columns="yearMth").fillna(0)
    df_pivot2 = df_pivot2.reset_index(drop=False)

    category = [c for c in sorted(set(df['HASH']))]
    #category = ['CAR','FOOD']
    colors = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78','#2ca02c','#98df8a','#d62728','#ff9896','#9467bd','#c5b0d5','#8c564b']
    # since yearMth is index in the pivot, I'm taking the values from the original df <redo>
    dates = [str(a) for a in sorted(set(df.yearMth)) if (a > b)]

    print dates
    wtf = {'time' : dates}

    for i in range(len(category)):
        name = df_pivot2.iloc[i:i+1,:1].get_value(i,'HASH')
        values = df_pivot2.iloc[i+0:i+1,1:].get_values()
        val = values[0,:].tolist()
        wtf.update({name : val})
    sourceW.data=wtf

print len(sourceW.data['time']), df.shape
update_data(201608)
print len(sourceW.data['time'])
p.x_range


26 (703, 12)
['201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709']
13


In [78]:
b = 201612
dates = [str(a) for a in sorted(set(df.yearMth)) if (a > b)]

print dates
print min(dates), max(dates)

['201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709']
201701 201709


In [ ]:
df.columns


In [ ]:
import matplotlib.pyplot as plt
plt.plot(df.Cost)
plt.show()

/Users/tomas/miniconda2/envs/py27_nb/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [97]:
import numpy as np
from bokeh.core.properties import value
from math import pi
b = 201508
df_tmp = df[(df['yearMth']>b) & (df['Cost']>0)]
df_pivot2 = df_tmp.pivot_table(values='Cost', index=['HASH'],aggfunc=np.sum, columns="yearMth").fillna(0)
df_pivot2 = df_pivot2.reset_index(drop=False)
df_pivot2
plot.line(df_pivot2[201509])

TypeError: line() takes exactly 3 arguments (2 given)

In [28]:
category = [c for c in sorted(set(df['HASH']))]
#category = ['CAR','FOOD']
colors = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78','#2ca02c','#98df8a','#d62728','#ff9896','#9467bd','#c5b0d5','#8c564b']
# since yearMth is index in the pivot, I'm taking the values from the original df <redo>
dates = [str(a) for a in sorted(set(df_tmp.yearMth)) if (a >= b)]

wtf = {'time' : dates}

for i in range(len(category)):
    name = df_pivot2.iloc[i:i+1,:1].get_value(i,'HASH')
    values = df_pivot2.iloc[i+0:i+1,1:].get_values()
    val = values[0,:].tolist()
    wtf.update({name : val})

#---------
sourceW = ColumnDataSource(data=wtf)
original_source = ColumnDataSource(df)

p = figure(x_range=dates, plot_height=500, title="by Year", toolbar_location=None, tools="")
p.vbar_stack(category, x='time', width=0.9, legend=[value(x) for x in category], color = colors, source=sourceW)
# format plot
p.xaxis.major_label_orientation = -pi/4
p.y_range.start = -1000
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_right"
p.legend.orientation = "vertical"

show(p)

In [27]:
help(source.update)

Help on method update in module bokeh.core.has_props:

update(self, **kwargs) method of bokeh.models.sources.ColumnDataSource instance
    Updates the object's properties from the given keyword arguments.
    
    Returns:
        None
    
    Examples:
    
        The following are equivalent:
    
        .. code-block:: python
    
            from bokeh.models import Range1d
    
            r = Range1d
    
            # set properties individually:
            r.start = 10
            r.end = 20
    
            # update properties together:
            r.update(start=10, end=20)



In [2]:
df['yearMth'] = df['dt'].map(lambda x: 100*x.year + x.month)
df['yearMth'] = df['dt'].map(lambda x: 100*x.year + x.month)

,Bokforingsdatum,Valutadatum,Verifikationsnummer,Text / mottagare,Belopp,Saldo,RND,LEFT(D6;4),HASH,dt,Cost,yearMth
689,2015-08-31,2015-08-30,5243900766S,OK-Q8 BANK AB,-2157.79,"71 531,62",1,OK-Q,CAR,2015-08-31,2157.79,201508
685,2015-08-31,2015-08-31,9900001100,COOP MEDMERA,-4950.38,"65 827,24",1,COOP,FOOD,2015-08-31,4950.38,201508
701,2015-08-28,2015-08-28,9900002100,DAGENS NYHET,-324.00,"71 154,41",0,DAGE,HOUSE,2015-08-28,324.00,201508
686,2015-08-31,2015-08-31,9900002100,NATURVETARNA,-240.00,"70 777,62",1,NATU,INSURANCE,2015-08-31,240.00,201508
688,2015-08-31,2015-08-30,5237900764S,AKADEMIKERNAS A-KASSA,-90.00,"71 441,62",1,AKAD,INSURANCE,2015-08-31,90.00,201508
690,2015-08-31,2015-08-30,5223900761S,HUDDINGE KOMMUN,-1458.00,"73 689,41",0,HUDD,KIDS,2015-08-31,1458.00,201508
687,2015-08-31,2015-08-30,5236900763S,TELE2 SVERIGE AB/COMVIQ,-424.00,"71 017,62",1,TELE,LINA,2015-08-31,424.00,201508
692,2015-08-28,2015-08-28,5490990250,LÅN 38801619,-1069.00,"75 669,41",0,LÅN,LOAN,2015-08-28,1069.00,201508
693,2015-08-28,2015-08-28,5490990250,LÅN 30664299,-733.00,"76 738,41",1,LÅN,LOAN,2015-08-28,733.00,201508
694,2015-08-28,2015-08-28,5490990250,LÅN 30664302,-800.00,"77 471,41",0,LÅN,LOAN,2015-08-28,800.00,201508


In [11]:
df['year'] = df['dt'].dt.year
df.groupby(['HASH', 'year'])['Cost'].sum()/12

HASH       year
CAR        2015      634.010000
           2016     1816.979167
           2017     1703.842500
FOOD       2015     3608.460833
           2016     5888.572500
           2017     4912.688333
HOUSE      2015     2337.435000
           2016     3620.044167
           2017     8568.701667
INSURANCE  2015      137.500000
           2016      542.000000
           2017      848.083333
KIDS       2015      777.166667
           2016     1534.333333
           2017     -501.500000
LANDET     2015      160.500000
           2016      475.500000
LINA       2015     -603.166667
           2016    -5486.000000
           2017     -679.750000
LOAN       2015    -1801.250000
           2016    -4910.166667
           2017    -3285.583333
OTHER      2015     3232.666667
           2016     3537.000000
           2017      826.916667
SHOPPING   2015      871.625000
           2016      902.029167
           2017     3513.486667
TOMAS      2015    -7039.833333
           2016    -8629

In [36]:
import numpy as np
aaa = pd.crosstab(df.HASH, df.year, df.Cost, aggfunc = "sum")/12
#aaa = aaa.reset_index(drop=True)
aaa

year,2015,2016,2017
HASH,,,
CAR,634.010000,1816.979167,1703.842500
FOOD,3608.460833,5888.572500,4912.688333
HOUSE,2337.435000,3620.044167,8568.701667
INSURANCE,137.500000,542.000000,848.083333
KIDS,777.166667,1534.333333,-501.500000
LANDET,160.500000,475.500000,NaN
LINA,-603.166667,-5486.000000,-679.750000
LOAN,-1801.250000,-4910.166667,-3285.583333
OTHER,3232.666667,3537.000000,826.916667


In [43]:
bbb = aaa.dropna()
bbb

year,2015,2016,2017
HASH,,,
CAR,634.010000,1816.979167,1703.842500
FOOD,3608.460833,5888.572500,4912.688333
HOUSE,2337.435000,3620.044167,8568.701667
INSURANCE,137.500000,542.000000,848.083333
KIDS,777.166667,1534.333333,-501.500000
LINA,-603.166667,-5486.000000,-679.750000
LOAN,-1801.250000,-4910.166667,-3285.583333
OTHER,3232.666667,3537.000000,826.916667
SHOPPING,871.625000,902.029167,3513.486667


In [19]:
from bokeh.models.widgets import PreText
stats = PreText(text='', width=200)
df['year'] = df['dt'].dt.year
stats = pd.crosstab(df.HASH, df.year, df.Cost, aggfunc = "sum")/12


In [86]:
# EXAMPLE DATA_TABLE FORMATTER
from bokeh import plotting
from bokeh import models 
from bokeh.models import widgets
import pandas, datetime, numpy
import bokeh
print(bokeh.__version__)



plotting.output_notebook()
df = pandas.DataFrame(dict(Date = [datetime.date(2014, 1,1), datetime.date(2014, 1,2), datetime.date(2014, 1,3)], Value = [1,2,3]))

df['Date'] = pandas.to_datetime(df['Date']) # annoyingly the way I built the data frame, pandas makes the "Date" column an object



source = models.ColumnDataSource(df)




columns = [

    widgets.TableColumn(field=c, title=c, formatter=models.DateFormatter(format='m/d/yy') if numpy.issubdtype(df[c].dtype, numpy.datetime64) else None) for c in df.columns
]
data_table = widgets.DataTable(source=source, columns=columns, editable=True, width = 500)
plotting.show(data_table)

0.12.7


Loading BokehJS ...

In [85]:
from bokeh.models import ColumnDataSource, TableColumn, DataTable, CustomJS, PrintfTickFormatter, DateFormatter

x = df['dt']
y = df['Belopp']

s_xy = ColumnDataSource(data=dict(x=x, y=y))
s_xy2 = ColumnDataSource(data=dict(x=[], y=[]))
##---
#datefmt = DateFormatter(format="ddMyy") 

columns = [
            TableColumn(field='x',  title="Date", formatter = DateFormatter(format='m/d/yy')),
            TableColumn(field='x',  title="Date2"),
            TableColumn(field='y',  title="Belopp")
]
data_table = DataTable(source=s_xy, columns=columns, editable = True)
##---
show(data_table)